In [1]:
#!pip install yfinance

In [2]:
#!pip install plotly

In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import polars as pl
import sklearn as sklearn
import scipy as scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import tensorflow.keras as keras
import tensorflow as tf
import cv2 as opencv
import numpy as np
import yfinance
import datetime as dt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Add, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import plotly
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error, explained_variance_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

2024-11-13 20:44:37.838132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


2024-11-13 20:44:40.026418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-13 20:44:40.050279: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-13 20:44:40.050416: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
def get_yf_hist(symbol_list,startDate,endDate,interval):
  import matplotlib.pyplot as plt
  import datetime
  import yfinance as yf

  # Get historical pricing data
  data = yf.download(symbol_list, startDate, endDate, interval)

  return data


def get_intraday(symbol_list,period,interval):
  intraday_list = []
  intraday_df = pd.DataFrame()
  for sym in symbol_list:
    intraday = yf.download(tickers=sym,
                           period=period,
                           interval=interval)
    intraday['symbol'] = sym
    intraday_df = pd.concat([intraday_df,intraday])

  return intraday_df

def rolling_zscore(data,return_period,window_length):
  log_returns = (np.log(data / data.shift(return_period)))
  zscore = (log_returns - log_returns.rolling(window_length).mean() / log_returns.rolling(window_length).std())
  #results_dict = dict({'log_returns':log_returns})
  results_df = pd.DataFrame(zscore)

  return results_df


  # convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def rmse(y_true,y_pred):
    
    # Ensure both lists have the same length
    if len(y_true) != len(y_pred):
        raise ValueError("The length of actual and predicted values must be the same")
    
    # Calculate the squared differences
    squared_diff = [(yt-yp) ** 2 for yt, yp in zip(y_true,y_pred)]
    
    # Calculate mean of the squared differences
    mean_squared_diff = sum(squared_diff) / len(y_true)
    
    # Return the square root of the mean squared difference
    rmse = math.sqrt(mean_squared_diff)
    
    return rmse

In [4]:
symbol_list = ['^GSPC']
startDate = '1900-01-01'
endDate = '2024-10-23'

close_df = get_yf_hist(symbol_list,startDate,endDate,interval='1d')

close_df = close_df['Adj Close']

close_df.head()

[*********************100%***********************]  1 of 1 completed


Date
1927-12-30    17.660000
1928-01-03    17.760000
1928-01-04    17.719999
1928-01-05    17.549999
1928-01-06    17.660000
Name: Adj Close, dtype: float64

In [5]:
close_df = close_df.reset_index()
close_df['Month'] = close_df['Date'].dt.month
close_df['Day'] = close_df['Date'].dt.day
close_df['Year'] = close_df['Date'].dt.year

close_df.head()

,Date,Adj Close,Month,Day,Year
0,1927-12-30,17.660000,12,30,1927
1,1928-01-03,17.760000,1,3,1928
2,1928-01-04,17.719999,1,4,1928
3,1928-01-05,17.549999,1,5,1928
4,1928-01-06,17.660000,1,6,1928


In [6]:
close_df = close_df.rename(columns={'Adj Close': 'Price'})
close_df.head()

,Date,Price,Month,Day,Year
0,1927-12-30,17.660000,12,30,1927
1,1928-01-03,17.760000,1,3,1928
2,1928-01-04,17.719999,1,4,1928
3,1928-01-05,17.549999,1,5,1928
4,1928-01-06,17.660000,1,6,1928


In [7]:
import pandas as pd

def backtest_strategy(df, month, day, position, holding_period, exit_criteria, exit_value, num_shares, start_date, end_date):
    """
    Backtests a trading strategy on a given DataFrame of stock prices.
    
    Parameters:
    - df: DataFrame containing 'Date', 'Price', 'Month', 'Day', 'Year' columns.
    - month: The month number for entering the trade.
    - day: The day number for entering the trade.
    - position: 'long' or 'short' to specify the position type.
    - holding_period: Number of days the stock is held before closing the trade.
    - exit_criteria: 'time' for time-based exit, 'percentage' for percentage-based exit.
    - exit_value: The exit condition value (holding days for 'time', percentage change for 'percentage').
    - num_shares: The number of shares to trade.
    - start_date: The start date for the backtest (YYYY-MM-DD format).
    - end_date: The end date for the backtest (YYYY-MM-DD format).
    
    Returns:
    - DataFrame with returns, cumulative returns in both dollar and percentage terms.
    """
    
    # Ensure 'Date' column is datetime type
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Filter by the start and end date
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Initialize lists to store results
    returns_dollar = []
    returns_percentage = []
    cumulative_dollar = []
    cumulative_percentage = []
    cumulative_dollar_return = 0
    cumulative_percentage_return = 0
    
    # Loop through each row to simulate the trades
    for i, row in df.iterrows():
        # Check if the row matches the given month and day
        if row['Month'] == month and row['Day'] == day:
            entry_price = row['Price']
            entry_date = row['Date']
            
            # Exit based on time
            if exit_criteria == 'time':
                exit_index = i + holding_period
                if exit_index < len(df):
                    exit_price = df.iloc[exit_index]['Price']
                    exit_date = df.iloc[exit_index]['Date']
                else:
                    continue  # Skip if we can't hold for the required period
                
            # Exit based on percentage change
            elif exit_criteria == 'percentage':
                for j in range(i + 1, len(df)):
                    exit_price = df.iloc[j]['Price']
                    price_change_percentage = ((exit_price - entry_price) / entry_price) * 100
                    
                    if position == 'long' and abs(price_change_percentage) >= exit_value:
                        exit_date = df.iloc[j]['Date']
                        break
                    elif position == 'short' and abs(price_change_percentage) >= exit_value:
                        exit_date = df.iloc[j]['Date']
                        break
                else:
                    continue  # Skip if no percentage threshold is met

            # Calculate the trade returns
            if position == 'long':
                trade_return_dollar = (exit_price - entry_price) * num_shares
                trade_return_percentage = ((exit_price - entry_price) / entry_price) * 100
            elif position == 'short':
                trade_return_dollar = (entry_price - exit_price) * num_shares
                trade_return_percentage = ((entry_price - exit_price) / entry_price) * 100
            
            # Update cumulative returns
            cumulative_dollar_return += trade_return_dollar
            cumulative_percentage_return += trade_return_percentage
            
            # Store the results
            returns_dollar.append(trade_return_dollar)
            returns_percentage.append(trade_return_percentage)
            cumulative_dollar.append(cumulative_dollar_return)
            cumulative_percentage.append(cumulative_percentage_return)
    
    # Add results back to DataFrame
    df['Trade Return ($)'] = pd.Series(returns_dollar, index=df.index[:len(returns_dollar)])
    df['Trade Return (%)'] = pd.Series(returns_percentage, index=df.index[:len(returns_percentage)])
    df['Cumulative Return ($)'] = pd.Series(cumulative_dollar, index=df.index[:len(cumulative_dollar)])
    df['Cumulative Return (%)'] = pd.Series(cumulative_percentage, index=df.index[:len(cumulative_percentage)])
    
    return df

# Example usage
# Assuming you have a DataFrame 'spy_df' with columns ['Date', 'Price', 'Month', 'Day', 'Year']
# Example call:
# result_df = backtest_strategy(spy_df, month=5, day=15, position='long', holding_period=10,
#                               exit_criteria='time', exit_value=10, num_shares=100,
#                               start_date='2010-01-01', end_date='2020-12-31')



In [21]:
trading_df = backtest_strategy(df=close_df,month=1,day=1,position='long',holding_period=10,exit_criteria='time',exit_value=10,num_shares=100,start_date=startDate,end_date=endDate)

In [22]:
trading_df

,Date,Price,Month,Day,Year,Trade Return ($),Trade Return (%),Cumulative Return ($),Cumulative Return (%)
0,1927-12-30,17.660000,12,30,1927,NaN,NaN,NaN,NaN
1,1928-01-03,17.760000,1,3,1928,NaN,NaN,NaN,NaN
2,1928-01-04,17.719999,1,4,1928,NaN,NaN,NaN,NaN
3,1928-01-05,17.549999,1,5,1928,NaN,NaN,NaN,NaN
4,1928-01-06,17.660000,1,6,1928,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
24314,2024-10-16,5842.470215,10,16,2024,NaN,NaN,NaN,NaN
24315,2024-10-17,5841.470215,10,17,2024,NaN,NaN,NaN,NaN
24316,2024-10-18,5864.669922,10,18,2024,NaN,NaN,NaN,NaN
24317,2024-10-21,5853.979980,10,21,2024,NaN,NaN,NaN,NaN


In [23]:
trading_df.tail()

,Date,Price,Month,Day,Year,Trade Return ($),Trade Return (%),Cumulative Return ($),Cumulative Return (%)
24314,2024-10-16,5842.470215,10,16,2024,NaN,NaN,NaN,NaN
24315,2024-10-17,5841.470215,10,17,2024,NaN,NaN,NaN,NaN
24316,2024-10-18,5864.669922,10,18,2024,NaN,NaN,NaN,NaN
24317,2024-10-21,5853.979980,10,21,2024,NaN,NaN,NaN,NaN
24318,2024-10-22,5851.200195,10,22,2024,NaN,NaN,NaN,NaN


In [24]:
close_df[(close_df['Month'] == 10)]

,Date,Price,Month,Day,Year
189,1928-10-01,21.360001,10,1,1928
190,1928-10-02,21.260000,10,2,1928
191,1928-10-03,21.190001,10,3,1928
192,1928-10-04,21.260000,10,4,1928
193,1928-10-05,21.219999,10,5,1928
...,...,...,...,...,...
24314,2024-10-16,5842.470215,10,16,2024
24315,2024-10-17,5841.470215,10,17,2024
24316,2024-10-18,5864.669922,10,18,2024
24317,2024-10-21,5853.979980,10,21,2024


In [25]:
trading_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24319 entries, 0 to 24318
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   24319 non-null  datetime64[ns]
 1   Price                  24319 non-null  float64       
 2   Month                  24319 non-null  int32         
 3   Day                    24319 non-null  int32         
 4   Year                   24319 non-null  int32         
 5   Trade Return ($)       0 non-null      object        
 6   Trade Return (%)       0 non-null      object        
 7   Cumulative Return ($)  0 non-null      object        
 8   Cumulative Return (%)  0 non-null      object        
dtypes: datetime64[ns](1), float64(1), int32(3), object(4)
memory usage: 1.4+ MB
